In [5]:
import pygame
import math
import random
pygame.init()

screen = pygame.display.set_mode((600, 400))
pygame.display.set_caption("Игра 2")

background = pygame.image.load("flower_back.jpg")
background = pygame.transform.scale(background, (600, 400))

bee_image = pygame.image.load("bee.jpg").convert()
bee_image = pygame.transform.scale(bee_image, (30, 30))
bee_image.set_colorkey((255, 255, 255))

hoop_width, hoop_height = 60, 60
flower_image = pygame.image.load("flower_2.jpg").convert()
flower_image = pygame.transform.scale(flower_image, (hoop_width,hoop_height))
flower_image.set_colorkey((255, 255, 255))


pygame.mixer.init()
pygame.mixer.music.load("whispers-of-the-breeze_92934.wav")
pygame.mixer.music.play(-1) 

score_sound = pygame.mixer.Sound("bee-single-voiced-short.wav")  

start_x, start_y = 100, 300
x, y = start_x, start_y
radius = 15

vx, vy = 0, 0
gravity = 0.5
power = 12
angle = 45
bounce_loss = 0.7
on_ground = True

hoop_x = random.randint(60, 600 - 60)
hoop_y = random.randint(60, 400 - 60)


font = pygame.font.Font(None, 24)

scored_flag = False
score = 0

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP:
                angle = min(angle + 5, 80)
            elif event.key == pygame.K_DOWN:
                angle = max(angle - 5, 10)
            elif event.key == pygame.K_w:
                power = min(power + 1, 25)
            elif event.key == pygame.K_s:
                power = max(power - 1, 5)
            elif event.key == pygame.K_SPACE and on_ground:
                rad = math.radians(angle)
                vx = power * math.cos(rad)
                vy = -power * math.sin(rad)
                on_ground = False

    if not on_ground:
        vy += gravity
        x += vx
        y += vy

    if y + radius >= 400:
        y = 400 - radius
        vy = -vy * bounce_loss
        vx *= bounce_loss
        if abs(vy) < 1 and abs(vx) < 1:
            x, y = start_x, start_y
            vx, vy = 0, 0
            on_ground = True
            scored_flag = False
            hoop_x = random.randint(60, 600 - 60)
            hoop_y = random.randint(60, 400 - 60)
    hoop_rect = pygame.Rect(hoop_x, hoop_y, hoop_width, hoop_height)
    bee_rect = pygame.Rect(x - radius, y - radius, radius * 2, radius * 2)

    if bee_rect.colliderect(hoop_rect) and not scored_flag:
        scored_flag = True
        score += 1
        score_sound.play()

    screen.blit(background, (0, 0))

    if on_ground:
        points = []
        rad = math.radians(angle)
        temp_vx = power * math.cos(rad)
        temp_vy = -power * math.sin(rad)
        temp_x, temp_y = start_x, start_y
        for i in range(60):
            temp_vy += gravity
            temp_x += temp_vx
            temp_y += temp_vy
            if temp_y + radius >= 400:
                break
            points.append((int(temp_x), int(temp_y)))
        if len(points) > 1:
            pygame.draw.lines(screen, (0, 0, 0), False, points, 2)
    
    
    screen.blit(bee_image, (int(x - radius), int(y - radius)))
    screen.blit(flower_image, (hoop_x, hoop_y))

    text_angle = font.render(f"Угол: {angle}°", True, (255, 0, 0))
    text_power = font.render(f"Сила: {power}", True, (255, 0, 0))
    text_score = font.render(f"Счёт: {score}", True, (255, 0, 0))
    screen.blit(text_angle, (10, 10))
    screen.blit(text_power, (10, 30))
    screen.blit(text_score, (10, 50))

    if scored_flag:
        hit_text = font.render("ПОПАДАНИЕ!", True, (0, 255, 0))
        screen.blit(hit_text, (250, 50))

    pygame.display.flip()
    pygame.time.delay(20)

pygame.quit()